
# Flowsheet CSTR Simulation and Optimization of Ethylene Glycol Production


## Learning outcomes


- Call and implement the IDAES CSTR unit model
- Construct a steady-state flowsheet using the IDAES unit model library
- Connecting unit models in a flowsheet using Arcs
- Fomulate and solve an optimization problem
    - Defining an objective function
    - Setting variable bounds
    - Adding additional constraints 


## Problem Statement

This example is adapted from Fogler, H.S., Elements of Chemical Reaction Engineering 5th ed., 2016, Prentice Hall,  p. 157-160.

Ethylene glycol (EG) is a high-demand chemical, with billions of pounds produced every year for applications such as vehicle anti-freeze. EG may be readily obtained from the hydrolysis of ethylene oxide in the presence of a catalytic intermediate. In this example, an aqueous solution of ethylene oxide hydrolizes after mixing with an aqueous solution of sulfuric acid catalyst:

**C<sub>2</sub>H<sub>4</sub>O</sub> + H<sub>2</sub>O</sub> + H<sub>2</sub>S</sub>O<sub>4</sub> → C<sub>2</sub>H<sub>6</sub>O<sub>2</sub> + H<sub>2</sub>S</sub>O<sub>4</sub>**

This reaction often occurs by two mechanisms, as the catalyst may bind to either reactant before the final hydrolysis step; we will simplify the reaction to a single step for this example.

The flowsheet that we will be using for this module is shown below with the stream conditions. We will be processing ethylene oxide and catalyst solutions of fixed concentrations to produce 200 MM lb/year of EG. As shown in the flowsheet, the process consists of a mixer M101 for the two inlet streams, a heater H101 to preheat the feed to the reaction temperature, and a CSTR unit R101 with zero overall duty (external cooling required for the exothermic reaction). We will assume ideal solutions and thermodynamics for this flowsheet, as well as well-mixed liquid behavior (no vapor phase) in the reactor. The properties required for this module are available in the same directory:

- egprod_ideal.py
- egprod_reaction.py

The state variables chosen for the property package are **molar flows of each component by phase in each stream, temperature of each stream and pressure of each stream**. The components considered are: **ethylene oxide, water, sulfuric acid and ethylene glycol** and the process occurs in liquid phase only. Therefore, every stream has 4 flow variables, 1 temperature and 1 pressure variable. 

![](egprod_flowsheet.png)


## Importing required pyomo and idaes components


To construct a flowsheet, we will need several components from the pyomo and idaes package. Let us first import the following components from Pyomo:
- Constraint (to write constraints)
- Var (to declare variables)
- ConcreteModel (to create the concrete model object)
- Expression (to evaluate values as a function of variables defined in the model)
- Objective (to define an objective function for optimization)
- SolverFactory (to solve the problem)
- TransformationFactory (to apply certain transformations)
- Arc (to connect two unit models)

For further details on these components, please refer to the pyomo documentation: https://pyomo.readthedocs.io/en/latest/

From idaes, we will be needing the FlowsheetBlock and the following unit models:
- Mixer
- Heater
- CSTR

We will also be needing some utility tools to put together the flowsheet and calculate the degrees of freedom.


In [1]:
from pyomo.environ import (Constraint,
                           exp,
                           Var,
                           ConcreteModel,
                           Expression,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value,
                           units as pyunits)
from pyomo.network import Arc, SequentialDecomposition

from idaes.core import FlowsheetBlock
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)
from idaes.generic_models.properties.core.generic.generic_reaction import (
        GenericReactionParameterBlock)
from idaes.generic_models.unit_models import (Mixer,
                                              Heater,
                                              CSTR)

from idaes.core.util.constants import Constants as c
from idaes.core.util import get_solver
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.initialization import (propagate_state,
                                            fix_state_vars,
                                            revert_state_vars)

import idaes.logger as idaeslog

## Importing required thermo and reaction package

The final set of imports are to import the thermo and reaction package. We have created a custom thermo package that support ideal vapor and liquid behavior for this system, and in this case we will restrict it to ideal liquid behavior only.

The reaction package here assumes Arrhenius kinetic behavior for the CSTR, for which $k_0$ and $E_a$ are known *a priori* (if unknown, they may be obtained using one of the parameter estimation tools within IDAES).

$ r = -kVC_{EO} $, $ k = k_0 e^{(-E_a/RT)}$, with the variables as follows:

$r$ - reaction rate extent in moles of ethylene oxide consumed per second; note that the traditional reaction rate would be given by $rate = r/V$ in moles per $m^3$ per second  
$k$ - reaction rate constant per second  
$V$ - volume of CSTR in $m^3$, note that this is *liquid volume* and not the *total volume* of the reactor itself  
$C_{EO}$ - bulk concentration of ethylene oxide in moles per $m^3$ (the limiting reagent, since we assume excess catalyst and water)  
$k_0$ - pre-exponential Arrhenius factor per second  
$E_a$ - reaction activation energy in kJ per mole of ethylene oxide consumed  
$R$ - gas constant in J/mol-K  
$T$ - reactor temperature in K

These calculations are contained within the property, reaction and unit model packages, and do not need to be entered into the flowsheet. More information on property estimation may be found below:

ParamEst parameter estimation: https://idaes-pse.readthedocs.io/en/stable/user_guide/workflow/data_rec_parmest.html?highlight=paramest  
HELMET thermodynamic estimation: https://idaes-pse.readthedocs.io/en/stable/user_guide/modeling_extensions/surrogate/helmet/index.html  
RIPE reaction estimation: https://idaes-pse.readthedocs.io/en/stable/user_guide/modeling_extensions/surrogate/ripe/index.html  

Let us import the following modules from the same directory as this jupyter notebook:
      <ul>
         <li>egprod_ideal as thermo_props</li>
         <li>egprod_reaction as reaction_props </li>
      </ul>
</div>

In [2]:
import egprod_ideal as thermo_props
import egprod_reaction as reaction_props

## Constructing the Flowsheet

We have now imported all the components, unit models, and property modules we need to construct a flowsheet. Let us create a ConcreteModel and add the flowsheet block. 

In [3]:
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})

We now need to add the property packages to the flowsheet. Unlike Module 1, where we only had a thermo property package, for this flowsheet we will also need to add a reaction property package. We will use the Generic Property and Generic Reaction Frameworks; more information may be found on these methods at https://idaes-pse.readthedocs.io/en/1.8.0/user_guide/components/property_package/index.html.

In [4]:
m.fs.thermo_params = GenericParameterBlock(default=thermo_props.config_dict)
m.fs.reaction_params = GenericReactionParameterBlock(default={"property_package": m.fs.thermo_params,
                                                              **reaction_props.config_dict})

## Adding Unit Models

Let us start adding the unit models we have imported to the flowsheet. Here, we are adding the Mixer (assigned a name M101) and a Heater (assigned a name H101). Note that all unit models need to be given a property package argument. In addition to that, there are several arguments depending on the unit model, please refer to the documentation for more details (https://idaes-pse.readthedocs.io/en/latest/model_libraries/core_lib/unit_models/index.html). For example, the Mixer unit model here is given a `list` consisting of names to the two inlets.

In [5]:
m.fs.M101 = Mixer(default={"property_package": m.fs.thermo_params,
                           "inlet_list": ["reagent_feed", "catalyst_feed"]})
m.fs.H101 = Heater(default={"property_package": m.fs.thermo_params,
                            "has_pressure_change": False,
                            "has_phase_equilibrium": False})

In [6]:
#Todo: Add reactor with the specifications above
m.fs.R101 = CSTR(
            default={"property_package": m.fs.thermo_params,
                     "reaction_package": m.fs.reaction_params,
                     "has_heat_of_reaction": True,
                     "has_heat_transfer": True,
                     "has_pressure_change": False})

## Connecting Unit Models using Arcs

We have now added all the unit models we need to the flowsheet. However, we have not yet specifed how the units are to be connected. To do this, we will be using the `Arc` which is a pyomo component that takes in two arguments: `source` and `destination`. Let us connect the outlet of the mixer(M101) to the inlet of the heater(H101). 

In [7]:
m.fs.s03 = Arc(source=m.fs.M101.outlet, destination=m.fs.H101.inlet)
m.fs.s04 = Arc(source=m.fs.H101.outlet, destination=m.fs.R101.inlet)

We have now connected the unit model block using the arcs. However, each of these arcs link to ports on the two unit models that are connected. In this case, the ports consist of the state variables that need to be linked between the unit models. Pyomo provides a convenient method to write these equality constraints for us between two ports and this is done as follows:

In [8]:
TransformationFactory("network.expand_arcs").apply_to(m)

## Adding expressions to compute operating costs

In this section, we will add a few Expressions that allows us to evaluate the performance. Expressions provide a convenient way of calculating certain values that are a function of the variables defined in the model. For more details on Expressions, please refer to: https://pyomo.readthedocs.io/en/latest/pyomo_modeling_components/Expressions.html

For this flowsheet, we are interested in computing ethylene glycol production in millions of pounds per year, as well as the total costs due to cooling and heating utilities:

Let us first add an Expression to convert the product flow from mol/s to MM lb/year of ethylene glycol. We see that our molecular weight exists in the thermo property package, so we may use that value for our calculations.

In [9]:
m.fs.eg_prod = Expression(expr=m.fs.R101.outlet.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"]*3600*24*365 # converting /s to /year
                          *m.fs.thermo_params.ethylene_glycol.mw # molecular weight from property package
                          *(2.205)/(1E6)) # convert from kg to MM lb

Now, let us add expressions to compute the reactor cooling cost (\\$/s) assuming a cost of 0.212E-4 \\$/kW, and the heating utility cost (\\$/s) assuming 2.2E-4 \\$/kW. Note that the heat duty is in units of watt (J/s). The total operating cost will be the sum of the two, expressed in \\$/year assuming 8000 operating hours per year.

In [10]:
m.fs.cooling_cost = Expression(expr=0.212e-7 * (-m.fs.R101.heat_duty[0]))  # the reaction is exothermic, so R101 duty is negative
m.fs.heating_cost = Expression(expr=2.2e-7 * m.fs.H101.heat_duty[0])  # the stream must be heated to T_rxn, so H101 duty is positive
m.fs.operating_cost = Expression(expr=(3600 * 8000 *(m.fs.heating_cost + m.fs.cooling_cost)))

## Fixing feed conditions

Let us first check how many degrees of freedom exist for this flowsheet using the `degrees_of_freedom` tool we imported earlier. We expect each stream to have 6 degrees of freedom, the mixer to have 0 (after both streams are accounted for), the heater to have 1 (just the duty, since the inlet is also the outlet of M101), and the reactor to have 1 (duty or conversion, since the inlet is also the outlet of H101). In this case, the reactor has an extra degree of freedom (reactor conversion or reactor volume) since we have not yet defined the CSTR performance equation.

In [11]:
print(degrees_of_freedom(m))  # (6 per stream x 2 streams) + (3 from units) = 15 DOF
print(degrees_of_freedom(m.fs.M101) - 12)  # 2 inlets, so subtract those 12 DOF
print(degrees_of_freedom(m.fs.H101) - 6)   # 1 inlet, so subtract those 6 DOF
print(degrees_of_freedom(m.fs.R101) - 6)   # 1 inlet, so subtract those 6 DOF

15
0
1
2


In [12]:
# Check the degrees of freedom
assert degrees_of_freedom(m) == 15
assert degrees_of_freedom(m.fs.M101) - 12 == 0
assert degrees_of_freedom(m.fs.H101) - 6 == 1
assert degrees_of_freedom(m.fs.R101) - 6 == 2

We will now be fixing the feed stream to the conditions shown in the flowsheet above. As mentioned in other tutorials, the IDAES framework expects a time index value for every referenced internal stream or unit variable, even in steady-state systems with a single time point $ t = 0 $. The non-present components in each stream are assigned a very small non-zero value to help with convergence and initializing. Based on stoichiometric ratios for the reaction, 80% conversion and 200 MM lb/year (46.4 mol/s) of ethylene glycol, we will initialize our simulation with the following calculated values:

In [13]:
m.fs.M101.reagent_feed.flow_mol_phase_comp[0, "Liq", "ethylene_oxide"].fix(58.0)
m.fs.M101.reagent_feed.flow_mol_phase_comp[0, "Liq", "water"].fix(39.6)  # calculated from 16.1 mol EO / cudm in stream
m.fs.M101.reagent_feed.flow_mol_phase_comp[0, "Liq", "sulfuric_acid"].fix(1e-5)
m.fs.M101.reagent_feed.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"].fix(1e-5)
m.fs.M101.reagent_feed.temperature.fix(298.15)
m.fs.M101.reagent_feed.pressure.fix(1e5)

m.fs.M101.catalyst_feed.flow_mol_phase_comp[0, "Liq", "ethylene_oxide"].fix(1e-5)
m.fs.M101.catalyst_feed.flow_mol_phase_comp[0, "Liq", "water"].fix(200)
m.fs.M101.catalyst_feed.flow_mol_phase_comp[0, "Liq", "sulfuric_acid"].fix(0.334)  # calculated from 0.9 wt% SA in stream
m.fs.M101.catalyst_feed.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"].fix(1e-5)
m.fs.M101.catalyst_feed.temperature.fix(298.15)
m.fs.M101.catalyst_feed.pressure.fix(1e5)

## Fixing unit model specifications

Now that we have fixed our inlet feed conditions, we will now be fixing the operating conditions for the unit models in the flowsheet. Let us initialize H101 with an outlet temperature of 328.15 K. 

In [14]:
m.fs.H101.outlet.temperature.fix(328.15)

We'll add constraints defining the reactor volume and conversion in relation to the stream properties. Particularly, we want to use our CSTR performance relation:  

$V = \frac{v_0 X} {k(1-X)}$, where the CSTR reaction volume $V$ will be specified, the inlet volumetric flow $v_0$ is determined by stream properties, $k$ is calculated by the reaction package, and $X$ will be calculated. Reactor volume is commonly selected as a specification in simulation problems, and choosing conversion is often to perform reactor design.

We will assume that 80% of our total reactor volume is working volume (CSTR reaction volume). For the CSTR, we have to define the conversion in terms of ethylene oxide as well as the CSTR reaction volume. This requires us to create new variables and constraints relating reactor properties to stream properties.  Note that the CSTR reaction volume variable (m.fs.R101.volume) does not need to be defined here since it is internally defined by the CSTR model. Additionally, the heat duty is not fixed, since the heat of reaction depends on the reactor conversion (through the extent of reaction and heat of reaction).

In [15]:
m.fs.R101.total_volume = Var(initialize=6.9225, bounds=(0, 15))  # m^3
m.fs.R101.conversion = Var(initialize=0.80, bounds=(0, 1))  # fraction

m.fs.R101.conv_constraint = Constraint(
    expr=m.fs.R101.conversion*m.fs.R101.inlet.
    flow_mol_phase_comp[0, "Liq", "ethylene_oxide"] ==
    (m.fs.R101.inlet.flow_mol_phase_comp[0, "Liq", "ethylene_oxide"] -
     m.fs.R101.outlet.flow_mol_phase_comp[0, "Liq", "ethylene_oxide"]))

m.fs.R101.vol_constraint = Constraint(
    expr=m.fs.R101.volume[0] == 0.80*m.fs.R101.total_volume)

m.fs.R101.rate_constant = Expression(
    expr=m.fs.reaction_params.reaction_R1.arrhenius_const *
    exp(-m.fs.reaction_params.reaction_R1.energy_activation /
        (c.gas_constant*m.fs.R101.control_volume.properties_in[0].temperature)))  # r = k[EO], k = [/s]

m.fs.R101.perf_eqn_constraint = Constraint(
    expr=m.fs.R101.volume[0] == (2*3.62e-3)*m.fs.R101.conversion /
    (m.fs.R101.rate_constant*(1 - m.fs.R101.conversion)))  # 2 streams, each with 3.62 cudm/s flow

m.fs.R101.total_volume.fix(6.9225)  # m^3

In [16]:
print(degrees_of_freedom(m))

0


In [17]:
# Check the degrees of freedom
assert degrees_of_freedom(m) == 0

Finally, we need to initialize the each unit operation in sequence to solve the flowsheet. In best practice, unit operations are initialized or solved, and outlet proeperties are propagated to connected inlet streams via arc definitions as follows:

In [18]:
# Initialize and solve each unit operation
m.fs.M101.initialize()
propagate_state(arc=m.fs.s03)

m.fs.H101.initialize()
propagate_state(arc=m.fs.s04)

m.fs.R101.initialize()

# set solver
solver = get_solver()

2021-11-03 14:26:23 [INFO] idaes.init.fs.M101.reagent_feed_state: Starting initialization
2021-11-03 14:26:23 [INFO] idaes.init.fs.M101.reagent_feed_state: Property initialization: optimal - Optimal Solution Found.
2021-11-03 14:26:23 [INFO] idaes.init.fs.M101.catalyst_feed_state: Starting initialization
2021-11-03 14:26:23 [INFO] idaes.init.fs.M101.catalyst_feed_state: Property initialization: optimal - Optimal Solution Found.
2021-11-03 14:26:23 [INFO] idaes.init.fs.M101.mixed_state: Starting initialization
2021-11-03 14:26:23 [INFO] idaes.init.fs.M101.mixed_state: Property initialization: optimal - Optimal Solution Found.
2021-11-03 14:26:23 [INFO] idaes.init.fs.M101.mixed_state: Property package initialization: optimal - Optimal Solution Found.
2021-11-03 14:26:23 [INFO] idaes.init.fs.M101: Initialization Complete: optimal - Optimal Solution Found
2021-11-03 14:26:23 [INFO] idaes.init.fs.H101.control_volume.properties_in: Starting initialization
2021-11-03 14:26:24 [INFO] idaes.ini

In [19]:
# Solve the model
results = solver.solve(m, tee=True)

Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale

In [20]:
# Check solver solve status
from pyomo.environ import TerminationCondition
assert results.solver.termination_condition == TerminationCondition.optimal

## Analyze the results of the square problem


What is the total operating cost? 

In [21]:
print('operating cost = $', value(m.fs.operating_cost), ' per year')

operating cost = $ 3451440.8331560493  per year


In [22]:
import pytest
assert value(m.fs.operating_cost)/1E6 == pytest.approx(3.451440, abs=1e-3)

For this operating cost, what conversion did we achieve of ethylene oxide to ethylene glycol? 

In [23]:
m.fs.R101.report()

print()
print('Conversion achieved = ', value(m.fs.R101.conversion)*100, '%')
print()
print('Total CSTR volume required = ', value(m.fs.R101.total_volume), 'm^3 = ', 264.172052*value(m.fs.R101.total_volume), ' gal')


Unit : fs.R101                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key       : Value       : Fixed : Bounds
    Heat Duty : -5.6457e+06 : False : (None, None)
       Volume :      5.5380 : False : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                                                 Inlet     Outlet  
    Molar Flowrate ('Liq', 'ethylene_oxide')      58.000     11.690
    Molar Flowrate ('Liq', 'water')               239.60     193.29
    Molar Flowrate ('Liq', 'sulfuric_acid')      0.33401    0.33401
    Molar Flowrate ('Liq', 'ethylene_glycol') 2.0000e-05     46.310
    Temperature                                   328.15     328.46
    Pressure                                  1.0000e+05 1.0000e+05

Conversion achieved =  79.84492859754936 %

Total CSTR volume r

In [24]:
assert value(m.fs.R101.conversion) == pytest.approx(0.79845, abs=1e-3)
assert value(m.fs.R101.volume[0]) == pytest.approx(5.5380, abs=1e-3)
assert value(m.fs.R101.heat_duty[0])/1E6 == pytest.approx(-5.6457, abs=1e-3)
assert value(m.fs.R101.outlet.temperature[0])/1E2 == pytest.approx(3.2846, abs=1e-3)
assert value(m.fs.R101.total_volume) == pytest.approx(6.9225, 1e-3)

## Optimizing Ethylene Glycol Production

Now that the flowsheet has been squared and solved, we can run a small optimization problem to minimize our production costs. Suppose we require at least 200 million pounds/year of ethylene glycol produced and at least 90% conversion of ethylene oxide, allowing for variable total reactor volume (considering operating/non-capital costs only) and reactor temperature (heater outlet).

Let us declare our objective function for this problem. 

In [25]:
m.fs.objective = Objective(expr=m.fs.operating_cost)

Now, we need to add the design constraints and unfix the decision variables as we had solved a square problem (degrees of freedom = 0) until now, as well as set bounds for the design variables:

In [26]:
m.fs.eg_prod_con = Constraint(expr=m.fs.eg_prod >= 200)  # MM lb/year
m.fs.conversion_con = Constraint(expr=m.fs.R101.conversion >= 0.90)  # fraction

m.fs.R101.total_volume.unfix()
m.fs.R101.total_volume.setlb(0)
m.fs.R101.total_volume.setub(18.93)  # 5000 gal

m.fs.H101.outlet.temperature.unfix()
m.fs.H101.outlet.temperature[0].setlb(328.15)
m.fs.H101.outlet.temperature[0].setub(470.45)  # highest component boiling point (ethylene glycol)

In [27]:
assert degrees_of_freedom(m) == 2


We have now defined the optimization problem and we are now ready to solve this problem. 




In [28]:
results = solver.solve(m, tee=True)

Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale

In [29]:
# Check for solver solve status
from pyomo.environ import TerminationCondition
assert results.solver.termination_condition == TerminationCondition.optimal

In [30]:
print('operating cost = $', value(m.fs.operating_cost), 'per year')

print()
print('Heater results')

m.fs.H101.report()

print()
print('CSTR reactor results')

m.fs.R101.report()

operating cost = $ 3889837.805924658 per year

Heater results

Unit : fs.H101                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key       : Value  : Fixed : Bounds
    Heat Duty : 699.26 : False : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                                                 Inlet     Outlet  
    Molar Flowrate ('Liq', 'ethylene_oxide')      58.000     58.000
    Molar Flowrate ('Liq', 'water')               239.60     239.60
    Molar Flowrate ('Liq', 'sulfuric_acid')      0.33401    0.33401
    Molar Flowrate ('Liq', 'ethylene_glycol') 2.0000e-05 2.0000e-05
    Temperature                                   298.15     328.15
    Pressure                                  1.0000e+05 1.0000e+05

CSTR reactor results

Unit : fs.R101                          

In [31]:
assert value(m.fs.operating_cost)/1E6 == pytest.approx(3.889837, abs=1e-3)
assert value(m.fs.R101.volume[0]) == pytest.approx(12.581, abs=1e-3)

Display optimal values for the decision variables and design variables:

In [32]:
print('Optimal Values')
print()

print('H101 outlet temperature = ', value(m.fs.H101.outlet.temperature[0]), 'K')

print()
print('Total CSTR volume required = ', value(m.fs.R101.total_volume), 'm^3 = ', 264.172052*value(m.fs.R101.total_volume), ' gal')

print()
print('Ethylene glycol produced = ', value(m.fs.eg_prod), 'MM lb/year')

print()
print('Conversion achieved = ', value(m.fs.R101.conversion)*100, ' %')

Optimal Values

H101 outlet temperature =  328.15 K

Total CSTR volume required =  15.72687246310235 m^3 =  4154.600170120042  gal

Ethylene glycol produced =  225.29973902961538 MM lb/year

Conversion achieved =  89.99999900004286  %


In [33]:
assert value(m.fs.H101.outlet.temperature[0])/100 == pytest.approx(3.2815, abs=1e-3)
assert value(m.fs.R101.total_volume) == pytest.approx(15.727, abs=1e-3)
assert value(m.fs.eg_prod) == pytest.approx(225.300, abs=1e-3)
assert value(m.fs.R101.conversion)*100 == pytest.approx(90.0, abs=1e-3)